## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
import os

### Reading th Data

In [2]:
PROJECT_DIR = r"C:\Users\khans\OneDrive\Desktop\AWS Project"
DATA_DIR = "data"

In [6]:
def get_data(name) :
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR,file_name )
    return pd.read_csv(file_path)

In [8]:
flights = get_data("flight_price")
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [9]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


### Preliminary Analysis

#### Check Data types

In [12]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

#### Checking for Duplicates

In [16]:
#flights.duplicated()
# Gives series of boolean values for all rows indicating weather they are duplicates or not
# index  False/true
flights.duplicated().sum()


220

In [21]:
(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(["Airline", "Date_of_Journey", "Source", "Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### Observations

- We can clearly see that "Date_of_Journey", "Dep_Time", "Arrival_Time" These sould be Date time object rather than String
- And "Duration" and "Total_Stops" should be of numeric Data Types
- 220 Duplicated rows need to be removed

### Detailed Analysis

In [50]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [73]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [53]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [52]:
(
    flights
    .Airline
    .str.replace("Premium economy", "")
    .str.replace("Business", "")
    .str.strip()
    .unique()
)

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

In [75]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [77]:
(
    flights.Dep_Time
    .loc[ lambda series: series.str.contains("[^0-9:]") ]
)

Series([], Name: Dep_Time, dtype: object)

In [79]:
pd.to_datetime(flights.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [89]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [150]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(-1).unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

In [158]:
(
    flights
    # .Arrival_Time
    .assign(Arrival_Time = lambda df : pd.to_datetime(df.Arrival_Time).dt.time)
    # .Arrival_Time.dt.time
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10:00,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15:00,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25:00,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30:00,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35:00,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25:00,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20:00,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20:00,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10:00,2h 40m,non-stop,No info,12648


In [90]:
pd.to_datetime(flights.Arrival_Time)

0       2025-03-22 01:10:00
1       2025-09-05 13:15:00
2       2025-06-10 04:25:00
3       2025-09-05 23:30:00
4       2025-09-05 21:35:00
                ...        
10678   2025-09-05 22:25:00
10679   2025-09-05 23:20:00
10680   2025-09-05 11:20:00
10681   2025-09-05 14:10:00
10682   2025-09-05 19:15:00
Name: Arrival_Time, Length: 10683, dtype: datetime64[ns]

In [91]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [108]:
(
     flights
    .Duration
    # .loc[lambda ser : ser.str.contains("[^hm0-9\s]")]
    .loc[lambda ser: ~ser.str.contains("m")].unique()
    
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [106]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [141]:
(
    flights
    .Duration
    .drop(6474)
    # .iloc[[6474]]
    .str.split(" ",expand = True)
    .set_axis(["hours", "minutes"], axis = 1)
    # .isna().sum()
    .assign(
        hours = lambda df_ : (  df_.
                              hours.
                              str.replace("h", "")
                              .astype("int")
                              .mul(60)
                             ),
        minutes = lambda df_ : (
            df_.
            minutes
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis = 1)
    .rename("duration_minute")
    .to_frame()
    .join(flights.Duration.drop(6474))
  
    # .isna().sum()
    
)

,duration_minute,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [143]:
def convert_to_minutes(ser) :
    return (
        ser
        .str.split(" ",expand = True)
        .set_axis(["hours", "minutes"], axis = 1)
        # .isna().sum()
        .assign(
            hours = lambda df_ : (  df_.
                                  hours.
                                  str.replace("h", "")
                                  .astype("int")
                                  .mul(60)
                                 ),
            minutes = lambda df_ : (
                df_.
                minutes
                .str.replace("m", "")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis = 1)
    )

In [175]:
(
    flights
    .Total_Stops
    .str.replace("non-stop","0")
    .str.replace(" stops?", "", regex = True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [178]:
(
    flights.
    Additional_Info.unique()
)

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

#### Data Cleaning

In [29]:
flights.select_dtypes(include = "object").columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

In [183]:
def clean_data(df):
    return (
        df
        .drop(index = 6474)
        .drop_duplicates()
        .assign(**{
            col : df[col].str.strip()
            for col in df.select_dtypes(include = "object").columns
        })
        # .rename(columns = str.lower )
        .rename(columns = lambda col : col.lower().strip() )
        .assign(
            airline = lambda df_ : (
                df_
                .airline
                .str.replace("Premium economy", "")
                .str.replace("Business", "")
                .str.strip()
                .str.title()
            ),
            source = lambda df_ : df_.source.str.replace("Banglore", "Bangalore"),
            destination = lambda df_ : df_.destination.str.replace("Cochin", "Cochi"),
            dep_time = lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration = lambda df_ : df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_ : (
                df_
                .total_stops
                .str.replace("non-stop","0")
                .str.replace(" stops?", "", regex = True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info = lambda df_ : df_.additional_info.str.replace("No info", "No Info")
        )
    )

In [184]:
cleaned_data = clean_data(flights)

In [48]:
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [185]:
cleaned_data.duplicated().sum()

2

In [186]:
(
    cleaned_data
    .loc[lambda ser : ser.duplicated()]
)

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
6805,Vistara,03/03/2019,Bangalore,New Delhi,BLR → DEL,21:10:00,00:05:00,175,0.0,No Info,7608
8733,Air Asia,24/03/2019,Bangalore,New Delhi,BLR → DEL,23:25:00,02:10:00,165,0.0,No Info,4482


### Split the Data

In [188]:
X = cleaned_data.drop(columns = "price")
Y = cleaned_data.price.copy()
Y

0         3897
1         7662
2        13882
3         6218
4        13302
         ...  
10678     4107
10679     4145
10680     7229
10681    12648
10682    11753
Name: price, Length: 10462, dtype: int64

In [189]:
X_,X_test,Y_,Y_test = train_test_split(X,Y, test_size = 0.2 ,random_state = 42)
X_train,X_val,Y_train,Y_val = train_test_split(X_,Y_, test_size = 0.2, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
print(X_val.shape, Y_val.shape)

(6695, 10) (6695,)
(2093, 10) (2093,)
(1674, 10) (1674,)


#### Export the Subsets

In [191]:
def export_data(x,y, name) :
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)

    x.join(y).to_csv(file_path, index = False)
    # when a data frame is saved to csv it makes a separate column of index that's whay index = false
    return pd.read_csv(file_path).head()

In [192]:
export_data(X_train, Y_train, "train")

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,21/03/2019,Bangalore,New Delhi,BLR → BOM → DEL,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,27/03/2019,Delhi,Cochi,DEL → BOM → COK,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,09/03/2019,Bangalore,New Delhi,BLR → DEL,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,12/06/2019,Kolkata,Banglore,CCU → BOM → BLR,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,12/03/2019,Bangalore,New Delhi,BLR → BOM → DEL,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087


In [193]:
export_data(X_val, Y_val, "val")

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,24/06/2019,Delhi,Cochi,DEL → BOM → COK,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,12/06/2019,Delhi,Cochi,DEL → HYD → COK,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,12/03/2019,Bangalore,New Delhi,BLR → BOM → DEL,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,6/06/2019,Delhi,Cochi,DEL → BOM → COK,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,18/05/2019,Delhi,Cochi,DEL → AMD → BOM → COK,23:05:00,04:25:00,1760,2.0,No Info,16704


In [194]:
export_data(X_test, Y_test, "test")

,airline,date_of_journey,source,destination,route,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,06/03/2019,Bangalore,New Delhi,BLR → BOM → DEL,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,6/06/2019,Kolkata,Banglore,CCU → BLR,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,18/03/2019,Kolkata,Banglore,CCU → BLR,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,27/06/2019,Chennai,Kolkata,MAA → CCU,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,6/05/2019,Kolkata,Banglore,CCU → BLR,15:15:00,17:45:00,150,0.0,No Info,4804
